In [2]:
import pyawr.mwoffice as mwo
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation 

'***PICK ONE***'
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
# %matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
%matplotlib tk 

In [3]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #Invoke circuit simulator analysis

In [4]:
graph_name = awrde.Project.Graphs.Item(2).Name
graph = awrde.Project.Graphs(graph_name)
meas = graph.Measurements[0]

num_pts = meas.XPointCount
xs = ys = dys = np.zeros(num_pts)

In [5]:
#Let's try to interpolate the regular-old S11 a bit better:
from scipy.interpolate import make_interp_spline, BSpline
from scipy.signal import savgol_filter
import scipy as sp

In [6]:
def animate_smooth(n_frm): #Animate method for FuncAnimation
    xs = np.asarray(meas.XValues)
    ys = np.asarray(meas.YValues(1))
    dys = np.gradient(ys, xs)
    abs_dys = np.abs(dys)

    new_xs = np.linspace(xs.min(), xs.max(), 100)

    #TODO: Choose appropriate k value for spline
    spl_ys = make_interp_spline(xs, ys, k=3)  # type: BSpline
    ys_smooth = spl_ys(new_xs)

    spl_dys1 = spl_ys.derivative()
    spl_dys2 = make_interp_spline(xs, dys, k=7)
    spl_abs_dys2 = make_interp_spline(xs, abs_dys, k=7)

    dys1_smooth = spl_dys1(new_xs)
    dys2_smooth = spl_dys2(new_xs)
    dys_savgol = savgol_filter(ys, window_length=99, polyorder=3, deriv=1)

    abs_dys1_smooth = np.abs(dys1_smooth)
    abs_dys2_smooth = spl_abs_dys2(new_xs)
    abs_dys_savgol = np.abs(dys_savgol)

    axs_smooth[0, 0].cla()
    axs_smooth[1, 0].cla()
    axs_smooth[2, 0].cla()
    axs_smooth[0, 0].plot(xs, ys, 'r.')
    axs_smooth[1, 0].plot(xs, dys, 'b.')
    axs_smooth[2, 0].plot(xs, abs_dys, 'g.')
    axs_smooth[0, 0].set_title('O.G.')

    axs_smooth[0, 1].cla()
    axs_smooth[1, 1].cla()
    axs_smooth[2, 1].cla()
    axs_smooth[0, 1].plot(new_xs, ys_smooth, 'r')
    axs_smooth[1, 1].plot(new_xs, dys2_smooth, 'b')
    axs_smooth[2, 1].plot(new_xs, abs_dys2_smooth, 'g')
    axs_smooth[0, 1].set_title('BSpline(Deriv)')
    
    axs_smooth[0, 2].cla()
    axs_smooth[1, 2].cla()
    axs_smooth[2, 2].cla()
    axs_smooth[0, 2].plot(new_xs, ys_smooth, 'r')
    axs_smooth[1, 2].plot(new_xs, dys1_smooth, 'b')
    axs_smooth[2, 2].plot(new_xs, abs_dys1_smooth, 'g')
    axs_smooth[0, 2].set_title('Deriv(BSpline)')

    axs_smooth[0, 3].cla()
    axs_smooth[1, 3].cla()
    axs_smooth[2, 3].cla()
    axs_smooth[0, 3].plot(xs, ys, 'r')
    axs_smooth[1, 3].plot(xs, dys_savgol, 'b')
    axs_smooth[2, 3].plot(xs, abs_dys_savgol, 'g')
    axs_smooth[0, 3].set_title('Sav-Gol')

In [7]:
# TODO: Get zoom to work?
fig_smooth, axs_smooth = plt.subplots(3, 4, sharex='all', sharey=False)


#TODO: Figure out this blit thing
ani = FuncAnimation(fig_smooth, animate_smooth, interval=200, blit=False) #Create animation object. blit=True is for smoother animations, only changed data should be updated.
plt.show()